# Реализация SSD
В это практическом уроке мы рассмотрим упрощённый пример реализации архитектуры SSD (Single Shot Multibox Detector). Цель урока -- разобраться с тем, как работает инференс в архитектуре SSD. Обучение SDD -- отдельный сложный вопрос, которы выходит за рамки данного урока.

### Загрузка необходимых библиотек
Здесь мы загружаем различне библиотеки, включая TensoFlow.

В TensorFlow инициируем режим жадного (eager) выполнения и проверяем версию (должна быть 1.14)

In [ ]:
import numpy as np

import tensorflow as tf
#tf.enable_eager_execution()
print('TensorFlow version:', tf.__version__)

### Модель SSD

Давайте реализуем класс, соответствующий модели SSD. Если предположить, что такая модель уже обучена, предсказание с её помощью сделать довольно просто.

Наша модель будет состоять из некоторого набора свёртчоных и пулинг слоёв, с помощью которых мы получаем несколько промежуточных карт признаков, соответствующих различным масштабам (`feat1`, `feat2`, `feat3`). Далее для каждого такого масштаба запускается детектор, который по сути является просто свёрточным слоем, задача которого предсказать классы и координаты боксов (всё это для каждого дефолт-бокса). Каждому пространственному пикселю тензора, который подаёдтся на вход в детектор соовтетствует несколько дефолт боксов (`num_def_boxes`), относительно которых мы и имщем объекты на картинке.

Итого, каждый детектор для одного пространственного пикселя карт признаков должен предсказать вектор размерности `N*(4+C)`, где N - кол-во дефолт боксов, C - кол-во классов.

Для наглядности можно раздеить каждый детектор на два параллельных свёрточных слоя: `conv_cls_i`, ответственный за классификацию дефолт-боксов (кол-во выходных каналов `N*C`), и `conv_loc_i`, ответственный за локализацию (кол-во выходных каналов `N*4`). 

То, как устроены дефолт-боксы (их расположение и размеры) имеет значение во время обучения, но не нужно для инференса. Нам нужно лишь знать, сколько дефолт-боксов есть в нашей модели.

В конце соединим предсказания со всех детекторов.

**[ЗАДАНИЕ 1]** Вопрос: Какое максимальное количество боксов может предсказать такая модель, если размер входной картинки равен будет 128x128, а num_def_boxes=3?

In [ ]:
class SSD(tf.keras.Model):
    def __init__(self, num_classes, num_def_boxes):
        super().__init__()
        self.num_classes = num_classes
 
        # Слои для извлечения признаков
        self.conv1 = tf.keras.layers.Conv2D(32, (5, 5), activation=tf.nn.relu, padding='same')
        self.conv2 = tf.keras.layers.Conv2D(32, (5, 5), activation=tf.nn.relu, padding='same')
        self.conv3 = tf.keras.layers.Conv2D(64, (5, 5), activation=tf.nn.relu, padding='same')
        self.conv4 = tf.keras.layers.Conv2D(64, (5, 5), activation=tf.nn.relu, padding='same')
        self.conv5 = tf.keras.layers.Conv2D(128, (5, 5), activation=tf.nn.relu, padding='same')
        self.conv6 = tf.keras.layers.Conv2D(128, (5, 5), activation=tf.nn.relu, padding='same')
        
        # Классификационные части детекторов (отдельный детектор для каждого масштаба)
        # Для каждого пикселя карт признаков предсказываются 
        # распределения вероятностей для всех дефолт боксов
        self.conv_cls1 = tf.keras.layers.Conv2D(num_def_boxes*num_classes, (3, 3), activation=tf.nn.relu, padding='same')
        self.conv_cls2 = tf.keras.layers.Conv2D(num_def_boxes*num_classes, (3, 3), activation=tf.nn.relu, padding='same')
        self.conv_cls3 = tf.keras.layers.Conv2D(num_def_boxes*num_classes, (3, 3), activation=tf.nn.relu, padding='same')
        
        # Локализационные части детекторов  (отдельный детектор для каждого масштаба)
        # Для каждого пикселя карт признаков предсказываются 
        # координаты всех дефолт боксов
        self.conv_loc1 = tf.keras.layers.Conv2D(num_def_boxes*4, (3, 3), activation=tf.nn.relu, padding='same')
        self.conv_loc2 = tf.keras.layers.Conv2D(num_def_boxes*4, (3, 3), activation=tf.nn.relu, padding='same')
        self.conv_loc3 = tf.keras.layers.Conv2D(num_def_boxes*4, (3, 3), activation=tf.nn.relu, padding='same')        
        
        self.pool = tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same')

    # Переход к тензору размера (batch, num_boxes, num_classes)
    # batch - кол-во образцов в батче
    # num_boxes - количество всех боксов для данного масштаба
    # num_classes - количество классов
    def reshape_cls(self, pred_cls):
        pred_cls = tf.transpose(pred_cls, (0, 3, 1, 2))
        pred_cls = tf.reshape(pred_cls, (pred_cls.shape[0], self.num_classes, -1))
        pred_cls = tf.transpose(pred_cls, (0, 2, 1))
        return pred_cls
      
    # Переход к тензору размера (batch, num_boxes, 4)
    # batch - кол-во образцов в батче
    # num_boxes - количество всех боксов для данного масштаба
    def reshape_loc(self, pred_loc):
        pred_loc = tf.transpose(pred_loc, (0, 3, 1, 2))
        pred_loc = tf.reshape(pred_loc, (pred_loc.shape[0], 4, -1))
        pred_loc = tf.transpose(pred_loc, (0, 2, 1))    
        return pred_loc
        
    def call(self, x):
        
        # Извлечение признаков
        out = self.conv1(x)
        out = self.conv2(out)
        feat1 = self.pool(out)
        out = self.conv3(feat1)
        out = self.conv4(out)
        feat2 = self.pool(out)
        out = self.conv5(feat2)
        out = self.conv6(out)
        feat3 = self.pool(out)
        
        # Применение детектора: классификационная часть
        pred_cls1 = self.conv_cls1(feat1)
        pred_cls2 = self.conv_cls2(feat2)
        pred_cls3 = self.conv_cls3(feat3)
        
        # Применение детектора: локализационная часть
        pred_loc1 = self.conv_loc1(feat1)
        pred_loc2 = self.conv_loc2(feat2)
        pred_loc3 = self.conv_loc3(feat3)
        
        # Для каждого масштаба переход к тензору размера (batch, num_boxes, num_classes)
        # в тензоре размера (batch, num_boxes, num_classes)
        pred_cls1 = self.reshape_cls(pred_cls1)
        pred_cls2 = self.reshape_cls(pred_cls2)
        pred_cls3 = self.reshape_cls(pred_cls3)
        
        # Для каждого масштаба получение тензора с координатами всех боксов 
        # в тензоре размера (batch, num_boxes, 4)
        pred_loc1 = self.reshape_loc(pred_loc1)
        pred_loc2 = self.reshape_loc(pred_loc2)
        pred_loc3 = self.reshape_loc(pred_loc3)
        
        # Объединение всех детекций для разнцх масштабов
        pred_cls = tf.concat([pred_cls1, pred_cls2, pred_cls3], axis=1)
        pred_loc = tf.concat([pred_loc1, pred_loc2, pred_loc3], axis=1)
                
        return pred_cls, pred_loc
        
model = SSD(num_classes=11, num_def_boxes=3)        

### Post-Processing
Сейчас наша SSD модель выдает ответы для всех возможных дефолт-боксов. В полной SSD архитектуре нужны две дополнительные стадии фильтрации: удаление боксов, соответствующих классу "фон" и удаление "дубликатов" с помощью метода Non-Maximum Suppression.

**[ЗАДАНИЕ 2]** Реализуйте первую фильтрацию предсказаний SSD модели -- чтобы остались только боксы, соответствующие объекту (нужно отбросить боксы, соответствующие классу "фон").



### Базовая модель
Архитектура SSD может быть реализована поверх любой произвольной модели CNN. Такая модель иногда называется "Базовая модель". Например, если есть CNN модель ResNet-101, то можно реализовать "SSD на основе ResNet-101".

**[ЗАДАНИЕ 3]** Ниже приведена реализация классификационной архитектуры VGG-16 (просто для ознакомления). Реализуйте архитектуру детектирования объектов SSD на основе VGG-16 (по аналогии с примеров в начале). Используйте 4 различных уровня (масштаба) признаков для детекторов (выберите соответствувющие тензоры самостоятельно).

In [ ]:
vgg = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(1000, activation='softmax'),
])